# To Do

1. Try different architectures
2. Try stateful/stateless LSTM.
3. Add OAT, holidays.
4. Check if data has consecutive blocks.

In [ ]:
import numpy as np
import pandas as pd
from scipy import stats
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.callbacks import EarlyStopping
from keras.layers import Dropout, Dense, LSTM
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

import matplotlib.pyplot as plt
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

In [ ]:
power_data_folder = '/Users/pranavhgupta/Documents/GitHub/XBOS_HVAC_Predictions/micro-service/data'
hvac_states_data_folder = '/Users/pranavhgupta/Documents/GitHub/XBOS_HVAC_Predictions/micro-service/hvac_states_batch_data'
site = 'avenal-animal-shelter'

# Import data

## Power data

In [ ]:
df_power = pd.read_csv(power_data_folder + '/power_' + site + '.csv', index_col=[0], parse_dates=True)
df_power.columns = ['power']
df_power.head()

In [ ]:
df_power.plot(figsize=(18,5))

### Check for missing data

In [ ]:
df_power.isna().any()

### Clean data

In [ ]:
# Resample to 5min
df_processed = df_power.resample('5T').mean()

df_processed.head()

In [ ]:
df_processed.plot(figsize=(18,5))

### Check for missing data

In [ ]:
print(df_processed.isna().any())
print('\n')
missing = df_processed['power'].isnull().sum()
total = df_processed['power'].shape[0]
print('% Missing data for power: ', (missing/total)*100, '%')

### Depending on the percent missing data, either drop it or forward fill the NaN's

In [ ]:
# Option 1: Drop NaN's
df_processed.dropna(inplace=True)

# # Option 2: ffill NaN's
# df_processed = df_processed.fillna(method='ffill')

### Normalize data

In [ ]:
scaler = MinMaxScaler(feature_range=(0,1))
df_normalized = pd.DataFrame(scaler.fit_transform(df_processed), 
                             columns=df_processed.columns, index=df_processed.index)
df_normalized.head()

### Check for missing data

In [ ]:
df_normalized.isna().any()

## Check for stationarity

In [ ]:
result = adfuller(df_normalized['power'], autolag='AIC')
output = pd.Series(result[0:4], index=['Test Statistic', 'p-value', '#Lags Used',
                                      '#Observations Used'])
for key, value in result[4].items():
    output['Critical Value (%s)' % key] = value
    
output

## HVAC States data

In [ ]:
df_hvac_states = pd.read_csv(hvac_states_data_folder + '/hvac_states_' + site + '.csv', 
                                 index_col=[0], parse_dates=True)
df_hvac_states.columns = ['zone' + str(i) for i in range(len(df_hvac_states.columns))]
df_hvac_states.head()

### Check for missing data

In [ ]:
df_hvac_states.isna().any()

### Convert categorical (HVAC states) into dummy variables

In [ ]:
var_to_expand = df_hvac_states.columns

# One-hot encode the HVAC states
for var in var_to_expand:

    add_var = pd.get_dummies(df_hvac_states[var], prefix=var, drop_first=True)

    # Add all the columns to the model data
    df_hvac_states = df_hvac_states.join(add_var)

    # Drop the original column that was expanded
    df_hvac_states.drop(columns=[var], inplace=True)
    
df_hvac_states.head()

In [ ]:
# def func(row):
#     """ Possible situations: (0,0,0), (1,0,1), (0,1,2) --> 0, 1, 2
    
#     If all are same --> first element
#     If there is a majority among the 3 --> majority
#     If all are unique --> last element
    
#     """

#     count = len(set(list(row.values)))
#     if count == 1:
#         return row.values[0]
#     elif count == 2:
#         max(set(list(row.values)), key=list(row.values).count)
#     else:
#         return row.values[-1]
    
# resample_df_hvac = df_raw_hvac_states.resample('15T').apply(func)

# resample_df_hvac = resample_df_hvac.fillna(method='ffill')
# resample_df_hvac.isna().any()

# Join power and hvac_states data

In [ ]:
# CHECK: pd.concat gives a lot of duplicate indices. 
# Try below code to see,
# start = pd.Timestamp('2018-02-10 06:00:00+00:00')
# df.loc[start]

df = pd.concat([df_normalized, df_hvac_states], axis=1)
df.head()

In [ ]:
df = df.drop_duplicates()

In [ ]:
missing = df.isnull().sum()
total = df.shape[0]
print('missing data for power: ', (missing/total)*100, '%')

### Depending on the percent missing data, either drop it or forward fill the NaN's

In [ ]:
# Option 1: Drop NaN's
df.dropna(inplace=True)

# # Option 2: ffill NaN's
# df = df.fillna(method='ffill')

# Visualizations

## Box plot

In [ ]:
df_box_plot = pd.DataFrame(df['power'])
df_box_plot['quarter'] = df_box_plot.index.quarter
df_box_plot.boxplot(column='power', by='quarter')

## Histogram

In [ ]:
df['power'].hist()

## ACF and PACF

In [ ]:
fig1 = plot_acf(df_processed['power'], lags=50)
fig2 = plot_pacf(df_processed['power'], lags=50)

# Prepare data

## Split into training & testing data

In [ ]:
X_train = df[(df.index < '2019-01-01')]
y_train = df.loc[(df.index < '2019-01-01'), 'power']

X_test = df[(df.index >= '2019-01-01')]
y_test = df.loc[(df.index >= '2019-01-01'), 'power']

## Prepare data for LSTM

Note: NUM_TIMESTEPS is a hyper-parameter too!

In [ ]:
# Number of columns in X_train
NUM_FEATURES    = len(X_train.columns)

# A sequence contains NUM_TIMESTEPS number of elements and predicts NUM_MODEL_PREDICTIONS number of predictions
NUM_TIMESTEPS   = 24

# Since this is an iterative method, model will predict only 1 timestep ahead
NUM_MODEL_PREDICTIONS = 1

# 4 hour predictions = Fourty eight 5min predictions
NUM_ACTUAL_PREDICTIONS = 48

In [ ]:
train_x, train_y = [], []
for i in range(NUM_TIMESTEPS, len(X_train)-NUM_MODEL_PREDICTIONS):
    train_x.append(X_train.values[i-NUM_TIMESTEPS:i])
    train_y.append(y_train.values[i:i+NUM_MODEL_PREDICTIONS]) 
train_x, train_y = np.array(train_x), np.array(train_y)
print(train_x.shape)
print(train_y.shape)

test_x, test_y = [], []
for i in range(NUM_TIMESTEPS, len(X_test)-NUM_MODEL_PREDICTIONS):
    test_x.append(X_test.values[i-NUM_TIMESTEPS:i])
    test_y.append(y_test.values[i:i+NUM_MODEL_PREDICTIONS])   
test_x, test_y = np.array(test_x), np.array(test_y)
print(test_x.shape)
print(test_y.shape)

# LSTM

In [ ]:
model = Sequential([
    
    LSTM(units=128, input_shape=(NUM_TIMESTEPS, NUM_FEATURES), return_sequences=True),
    Dropout(0.2),
    
    LSTM(units=128, return_sequences=True),
    Dropout(0.2),
    
    LSTM(units=128, activation='softmax', return_sequences=False),
    Dropout(0.2),
    
    Dense(NUM_MODEL_PREDICTIONS)
])

model.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])
model.summary()

In [ ]:
# Stop training if validation loss fails to decrease
callbacks = [EarlyStopping(monitor='val_loss', mode='min', verbose=1)]

history = model.fit(train_x, train_y, 
                    epochs=100, batch_size=128, shuffle=False, 
                    validation_data=(test_x, test_y), callbacks=callbacks)

# Results

## Loss

In [ ]:
train_loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = [x for x in range(len(train_loss))]

df_train_loss = pd.DataFrame(train_loss, columns=['train_loss'], index=epochs)
df_val_loss   = pd.DataFrame(val_loss, columns=['val_loss'], index=epochs)

df_loss = pd.concat([df_train_loss, df_val_loss], axis=1)

In [ ]:
df_loss.plot(figsize=(18,5))

## Accuracy

In [ ]:
train_acc = history.history['acc']
val_acc   = history.history['val_acc']
epochs    = [x for x in range(len(train_acc))]

df_train_acc = pd.DataFrame(train_acc, columns=['train_acc'], index=epochs)
df_val_acc   = pd.DataFrame(val_acc, columns=['val_acc'], index=epochs)

df_acc = pd.concat([df_train_acc, df_val_acc], axis=1)

In [ ]:
df_acc.plot(figsize=(18,5))

# Plot predicted & true values

In [ ]:
# Make predictions through trained model
pred_y = model.predict(test_x)

# Convert predicted and actual values to dataframes (for plotting)
df_y_pred = pd.DataFrame(scaler.inverse_transform(pred_y),
                         index=y_test[NUM_TIMESTEPS:-NUM_MODEL_PREDICTIONS].index, 
                         columns=['power'])

df_y_true = pd.DataFrame(scaler.inverse_transform(test_y),
                         index=y_test[NUM_TIMESTEPS:-NUM_MODEL_PREDICTIONS].index, 
                         columns=['power'])

In [ ]:
df_y_pred.head()

In [ ]:
df_plot = pd.concat([df_y_pred, df_y_true], axis=1)
df_plot.columns = ['pred', 'true']

df_plot.head()

In [ ]:
df_plot.plot(figsize=(18,5))

In [ ]:
# # Plot between two time periods

# start = pd.Timestamp('2019-01-01 23:45:00+00:00')
# end = pd.Timestamp('2019-02-01 23:45:00+00:00')
# df_plot.loc[start:end].plot(figsize=(18,5))

# Make predictions through iterative fitting for a particular timestamp

## Choose a particular timestamp

In [ ]:
timestamp = pd.Timestamp('2019-01-01 23:45:00+00:00')

# Keep copy of timestamp to use it after the for loop
orig_timestamp = timestamp

In [ ]:
X_test_pred = X_test.copy()

for _ in range(NUM_ACTUAL_PREDICTIONS):
    
    # Create test sequence
    test = np.array(X_test_pred.loc[:timestamp].tail(NUM_TIMESTEPS))
    test = np.reshape(test, (1, test.shape[0], test.shape[1]))
    
    # Increment timestamp
    timestamp = X_test_pred.loc[timestamp:].index.values[1]

    # Make prediction
    y_pred_power = model.predict(test)
    y_pred_power = list(y_pred_power[0])
    
    # Add prediction to end of test array
    X_test_pred.loc[timestamp, 'power'] = y_pred_power

In [ ]:
# X_test_pred.loc[pd.Timestamp('2019-01-01 23:45:00+00:00'):].head(NUM_ACTUAL_PREDICTIONS)

In [ ]:
# X_test.loc[pd.Timestamp('2019-01-01 23:45:00+00:00'):].head(NUM_ACTUAL_PREDICTIONS)

## Plot

In [ ]:
arr_pred = np.reshape(X_test_pred.loc[orig_timestamp:,'power'].head(NUM_ACTUAL_PREDICTIONS).values, (-1, 1))
arr_true = np.reshape(X_test.loc[orig_timestamp:,'power'].head(NUM_ACTUAL_PREDICTIONS).values, (-1, 1))

df_pred = pd.DataFrame(scaler.inverse_transform(arr_pred),
                         index=X_test_pred.loc[orig_timestamp:].head(NUM_ACTUAL_PREDICTIONS).index)

df_true = pd.DataFrame(scaler.inverse_transform(arr_true),
                         index=X_test.loc[orig_timestamp:].head(NUM_ACTUAL_PREDICTIONS).index)

In [ ]:
df_plot = pd.concat([df_pred, df_true], axis=1)
df_plot.columns = ['pred', 'true']

In [ ]:
df_plot.plot(figsize=(18,5))

# Get accuracy and mse of the entire test set using iterative fitting

Note: This takes a while to compute!

In [ ]:
# These two lists store the entire dataframes of 48 predictions of each element in test set!
# This is not really necessary but only to double check if the outputs are in the correct format
predicted_values = []
true_values = []

for i in range(NUM_TIMESTEPS, len(X_test)-NUM_ACTUAL_PREDICTIONS):
    
    # Keep copy of timestamp to store it for use after the for loop 
    timestamp = pd.Timestamp(X_test.index.values[i])
    orig_timestamp = timestamp
    
    X_test_pred = X_test.copy()
    
    for _ in range(NUM_ACTUAL_PREDICTIONS):
    
        # Create test sequence
        test = np.array(X_test_pred.loc[:timestamp].tail(NUM_TIMESTEPS))
        test = np.reshape(test, (1, test.shape[0], test.shape[1]))

        # Increment timestamp
        timestamp = X_test_pred.loc[timestamp:].index.values[1]

        # Make prediction
        y_pred_power = model.predict(test)
        y_pred_power = list(y_pred_power[0])

        # Add prediction to end of test array
        X_test_pred.loc[timestamp, 'power'] = y_pred_power
        
    predicted_values.append(X_test_pred.loc[orig_timestamp:].head(NUM_ACTUAL_PREDICTIONS))
    true_values.append(X_test.loc[orig_timestamp:].head(NUM_ACTUAL_PREDICTIONS))

In [ ]:
# Get only the power values from the original predicted_values and true_values lists and then reshape them 
# into the correct format for sklearn metrics' functions.

predicted_power_values = []
true_power_values = []

for df in predicted_values:
    predicted_power_values.append(df[['power']].values)
    
for df in true_values:
    true_power_values.append(df[['power']].values)
    
predicted_power_values = np.array(predicted_power_values)
predicted_power_values = np.reshape(predicted_power_values, 
                                    (predicted_power_values.shape[0], predicted_power_values.shape[1]))

true_power_values = np.array(true_power_values)
true_power_values = np.reshape(true_power_values, 
                                    (true_power_values.shape[0], true_power_values.shape[1]))

In [ ]:
from sklearn.metrics import r2_score
score = r2_score(true_power_values, predicted_power_values)
score

In [ ]:
from sklearn.metrics import mean_squared_error
mse = mean_squared_error(true_power_values, predicted_power_values)
mse